In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
from __init__ import *
from __init__jupyterlab import *

import snmcseq_utils

In [2]:
df_genes = pd.read_csv(PATH_GENEBODY_ANNOTATION, sep='\t')
df_genes['gid'] = df_genes['gene_id'].apply(lambda x: x.split('.')[0])
print(df_genes.shape)
df_genes.head()

(53379, 8)


,gene_id,gene_name,chr,start,end,strand,gene_type,gid
0,ENSMUSG00000102693.1,4933401J01Rik,chr1,3073253,3074322,+,TEC,ENSMUSG00000102693
1,ENSMUSG00000064842.1,Gm26206,chr1,3102016,3102125,+,snRNA,ENSMUSG00000064842
2,ENSMUSG00000051951.5,Xkr4,chr1,3205901,3671498,-,protein_coding,ENSMUSG00000051951
3,ENSMUSG00000102851.1,Gm18956,chr1,3252757,3253236,+,processed_pseudogene,ENSMUSG00000102851
4,ENSMUSG00000103377.1,Gm37180,chr1,3365731,3368549,-,TEC,ENSMUSG00000103377


In [4]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_clusterings.tsv'
df_clst = pd.read_csv(f, sep='\t')
print(df_clst.shape)
df_clst.head()

(408885, 11)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1
1,snmcseq_gene_2C_M_1,-2.414973,-12.098081,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
2,snmcseq_gene_2C_M_100,-1.446751,-12.051894,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1
3,snmcseq_gene_2C_M_1000,-1.901734,-13.660591,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
4,snmcseq_gene_2C_M_1001,-0.981086,-11.936621,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1


In [5]:
f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/10x_cells_v2_raw.npz'
fgene = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/10x_cells_v2_raw.gene'
fcell = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/10x_cells_v2_raw.cell'
gxc_mat = snmcseq_utils.load_gc_matrix(fgene, fcell, f)
gxc_mat.data.shape

(31053, 121440)

In [10]:
cluster_level = 'joint_cluster_round3'
df_sum = pd.DataFrame(index=gxc_mat.gene)
for clst, df_sub in df_clst.groupby(cluster_level):
    print(clst)
    cells = df_sub['sample'].values
    cells_idx = snmcseq_utils.get_index_from_array(gxc_mat.cell, cells)
    df_sum[clst] = np.ravel(gxc_mat.data.tocsc()[:, cells_idx].sum(axis=1))
    
# df_sum2 = pd.DataFrame(index=gxc_mat.gene)
# for clst, df_sub in df_clst.groupby('cluster_label'):
#     print(clst)
#     cells = df_sub['cell'].values
#     cells_idx = snmcseq_utils.get_index_from_array(gxc_mat.cell, cells)
#     df_sum2[clst] = np.ravel(gxc_mat.data.tocsc()[:, cells_idx].sum(axis=1))

1-1-1
1-2-1
1-2-2
10-1-1
2-1-1
2-1-2
2-2-1
2-2-2
2-2-3
2-2-4
2-3-1
2-3-2
2-3-3
3-1-1
3-1-2
3-2-1
3-2-2
3-3-1
4-1-1
4-1-2
4-2-1
4-2-2
4-2-3
5-1-1
5-1-2
5-2-1
5-2-2
5-2-3
5-3-1
5-3-2
5-4-1
5-5-1
6-1-1
6-1-2
6-2-1
6-3-1
6-3-2
6-3-3
6-4-1
6-4-2
6-5-1
6-5-2
6-5-3
6-6-1
7-1-1
7-1-2
7-2-1
7-2-2
8-1-1
8-1-2
8-2-1
8-3-1
8-3-2
9-1-1
9-2-1
9-2-2


In [11]:
f = '/cndd2/fangming/scf_enhancers/data/bulk/round3/rna/genecounts_10x_cells_v2.tsv'
df_sum.to_csv(f, sep="\t", index=True, header=True)

In [12]:
print(df_sum.shape)
df_sum.head()

(31053, 56)


,1-1-1,1-2-1,1-2-2,10-1-1,2-1-1,2-1-2,2-2-1,2-2-2,2-2-3,2-2-4,...,7-2-1,7-2-2,8-1-1,8-1-2,8-2-1,8-3-1,8-3-2,9-1-1,9-2-1,9-2-2
ENSMUSG00000051951,1459727,615259,64475,17852,387066,359025,627440,22186,10334,1940,...,78387,52060,34727,27376,50214,3591,1317,47597,13856,11192
ENSMUSG00000089699,3188,2484,513,168,1246,1008,2670,142,50,13,...,162,158,204,158,365,14,8,34,14,7
ENSMUSG00000102343,71,30,3,2,20,9,39,3,1,0,...,2,3,1,1,3,0,0,0,4,2
ENSMUSG00000025900,173,139,32,5,48,45,130,2,4,0,...,6,3,10,10,12,2,0,2,3,2
ENSMUSG00000025902,1,0,0,0,0,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
